In [ ]:
import numpy as np
import pandas as pd
import os
import time
import json
from pandas.io.json import json_normalize
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### 第一步，我们先把json格式展开

In [ ]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
train = load_df('/home/leechh/data/R/train.csv')

In [ ]:
train.head(5)

In [ ]:
def dfbar(df, feature, top=-1):
    value_count1 = train[feature].value_counts(dropna=True,normalize=True)
    plt.figure(figsize=[15,5])
    value_count1[0:top].plot(kind='bar',grid=True)
    plt.title(f'Group by {feature},{sum(train[feature].isnull())} NaN data')
    plt.show()
    return None


### channelGrouping 

In [ ]:
channel = train.channelGrouping.value_counts(dropna=True,normalize=True)
plt.figure(figsize=[15,5])
plt.subplot(1,2,1)
plt.barh(np.arange(channel.shape[0])+1,
         channel.values,
         alpha=0.3,
         facecolor='k',edgecolor='b',
         tick_label=channel.index.values)
plt.title(f'Group by channelGrouping,{sum(train.channelGrouping.isnull())} NaN data')

plt.subplot(1,2,2)
idx = train.totals_transactionRevenue != 0 
channel2 = train[idx].channelGrouping.value_counts(dropna=True,normalize=False)
plt.barh(np.arange(channel2.shape[0])+1,
         channel2.values,
         alpha=0.3,
         facecolor='k',edgecolor='b',
         tick_label=channel.index.values)
plt.title(f'Group by channelGrouping,{sum(train.channelGrouping.isnull())} NaN data')

plt.show()

In [ ]:
idx = train.totals_transactionRevenue != 0 


### 我觉得seesionId 应该就是fullvisitId 与 visitId 的结合，验证以下，如果是的话，我们就可以把 这个features去掉了。

In [ ]:
print('fullvisitId sess diff:',sum(train.sessionId.str.split('_',expand=True)[0] != train.fullVisitorId))
print('visitId sess diff:',sum(train.sessionId.str.split('_',expand=True)[1] != train.visitId.astype('str')))

In [ ]:
train.drop(['sessionId'],axis=1,inplace=True)

### 接下来，把时间date转化为时间格式

In [ ]:
train['date'] = pd.to_datetime(train.date,format='%Y%m%d')

### socialEngagementType

In [ ]:
idx = (train.socialEngagementType != 'Not Socially Engaged')
sum(idx)

In [ ]:
train.drop(['socialEngagementType'], axis=1, inplace=True)

In [ ]:
train.head()

### visitNumber

In [ ]:
dfbar(train,'visitNumber',top=30)

我们可以看到，visitNumber及其不平衡，大多数为1，由kaggle官方提供的说明，If this is the first session, then this is set to 1，所以大多数都是用户初次访问

### visitStartTime
The timestamp (expressed as POSIX time)， so we convert it to a pd.datatime

In [ ]:
def totime(t):
    year = time.localtime(t)[0]
    month = time.localtime(t)[1]
    day = time.localtime(t)[2]
    hour = time.localtime(t)[3]
    minute = time.localtime(t)[4]
    second = time.localtime(t)[5]
    return pd.Timestamp(year,month,day,hour,minute,second)


train['visitStartTime'] = train.visitStartTime.map(totime)

### 去掉重复features

In [ ]:
for i in train.columns:
    if sum(train[i] == train[i][0]) == train.shape[0]:
        train.drop([i], axis=1, inplace=True)

In [ ]:
train.shape

### device

In [ ]:
train.iloc[:,6:10].head()

In [ ]:
# 我们去掉device_isMobile
train.drop(['device_isMobile'], axis=1, inplace=True)

我们可以看到device_operatingSystem 有较多(not set)项目 ，我们前期可以先去掉这些数据，在后期可以用机器学习的方法找到其真实值

In [ ]:
dfbar(train, 'device_browser')

In [ ]:
plt.figure(figsize=[15,5])
value_count2 = train['device_deviceCategory'].value_counts(dropna=False,normalize=False)
value_count3 = train['device_operatingSystem'].value_counts(dropna=False,normalize=False)
plt.subplot(1,2,1)
value_count2.plot(kind='bar')
plt.subplot(1,2,2)
value_count3.plot(kind='bar')
plt.show()

### geoNetwork

In [ ]:
train.iloc[:,9:16].head()

In [ ]:
n = 1
for i in train.columns[9:16]:
    plt.subplot(7,1,n)
    value_count1 = train[i].value_counts(dropna=True,normalize=True)
    value_count1[0:10].plot(kind='barh',grid=True,figsize=[8,30])
    plt.title(f'Group by {i},{sum(train[i].isnull())} NaN data')
    n = n + 1
plt.show()


In [ ]:
train.fullVisitorId.nunique()

### totals

In [ ]:
train.iloc[:,16:21].head()

In [ ]:
train['totals_bounces'] = train.totals_bounces.fillna(0)
train['totals_newVisits'] = train.totals_newVisits.fillna(0)
train['totals_transactionRevenue'] = train.totals_transactionRevenue.fillna(0)

In [ ]:
plt.figure(figsize=[15,5])
plt.subplot(1,2,1)
bounces = train.totals_bounces.value_counts(normalize=True)
plt.bar(np.arange(bounces.shape[0]),
        bounces.values,
        width=0.2,
        alpha=0.3,
        facecolor='K',
        edgecolor='b',
        lw=1)
plt.title('Bounces count')
plt.ylabel('Percentage')

plt.subplot(1,2,2)
idx = train.totals_transactionRevenue != 0 
bounces = train.totals_bounces[idx].value_counts(normalize=True)
plt.bar(np.arange(bounces.shape[0]),
        bounces.values,
        width=0.2,
        alpha=0.3,
        facecolor='K',
        edgecolor='b',
        lw=1)
plt.title('Bounces count have Revenue')
plt.ylabel('Percentage')



In [ ]:
bounces = train.totals_bounces.value_counts(normalize=True)
bounces.index.values

In [ ]:
for i in train.columns[16:21]:
    dfbar(train,i,top=30)

### trafficSource

In [ ]:
train.iloc[:,21:].head()

In [ ]:
for i in train.columns[21:35]:
    print('\n',train.groupby(i).size())

In [ ]:
train.head()

In [ ]:
sum(train['trafficSource_adwordsClickInfo.isVideoAd'].isnull())